In [3]:
pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.5 MB/s eta 0:00:00


In [7]:
import subprocess, threading, queue, json

event_queue = queue.Queue()

def stm32_listener():
    proc = subprocess.Popen(
        ["python", "virtual_stm32.py"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    for line in proc.stdout:
        try:
            evt = json.loads(line.strip())
            event_queue.put(evt)
        except Exception:
            continue

threading.Thread(target=stm32_listener, daemon=True).start()

def process_events(conn):
    conn = sqlite3.connect("simulation.db")
    cur = conn.cursor()
    while not event_queue.empty():
        evt = event_queue.get()
        ts = evt.get("ts")
        event = evt.get("event")
        payload = evt.get("payload")
        cur.execute("INSERT INTO stm32_events(ts,event,payload) VALUES (?,?,?)",
                    (ts, event, json.dumps(payload)))

        # React to "device_present"
        if event == "device_present":
            model = "STM32-USB"
            cap = 32
            insert_device(conn, model, cap)   # reuse insert_device()
        conn.commit()


In [11]:
import sqlite3, heapq

DB_NAME = "simulation.db"

# ---------------- Database Functions ----------------
def create_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS events (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        time INTEGER NOT NULL,
        status TEXT NOT NULL
    )
    ''')
    conn.commit()
    conn.close()

def insert_event(name, time, status):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO events (name, time, status) VALUES (?, ?, ?)", (name, time, status))
    conn.commit()
    conn.close()

def fetch_events():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM events ORDER BY time ASC")
    rows = cursor.fetchall()
    conn.close()
    return rows

# ---------------- Event Class ----------------
class Event:
    def __init__(self, name, time):
        self.name = name
        self.time = time
        self.status = "Scheduled"

    def execute(self):
        self.status = "Completed"
        return f"Event {self.name} executed at time {self.time}"

# ---------------- Simulation Class ----------------
class Simulation:
    def __init__(self):
        self.current_time = 0
        self.event_queue = []

    def schedule_event(self, event):
        heapq.heappush(self.event_queue, (event.time, event))
        insert_event(event.name, event.time, event.status)

    def process_events(self):
        while self.event_queue:
            time, event = heapq.heappop(self.event_queue)
            self.current_time = time
            result = event.execute()
            insert_event(event.name, event.time, event.status)
            print(result)

# ---------------- Run Simulation ----------------
if __name__ == "__main__":
    create_database()

    sim = Simulation()
    sim.schedule_event(Event("Start Process", 5))
    sim.schedule_event(Event("Check Status", 10))
    sim.schedule_event(Event("Finish Process", 15))

    sim.process_events()

    print("\n--- Events Stored in Database ---")
    for row in fetch_events():
        print(row)


Event Start Process executed at time 5
Event Check Status executed at time 10
Event Finish Process executed at time 15

--- Events Stored in Database ---
(1, 'Start Process', 5, 'Scheduled')
(4, 'Start Process', 5, 'Completed')
(2, 'Check Status', 10, 'Scheduled')
(5, 'Check Status', 10, 'Completed')
(3, 'Finish Process', 15, 'Scheduled')
(6, 'Finish Process', 15, 'Completed')
